In [1]:
import cv2
import pandas as pd

In [5]:
def main(video_path, output_path, stats_output_path):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Get the video frame width, height, frames per second (fps), and total frame count
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # You can use 'XVID', 'MJPG', etc.
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    # Initialize a list to store motion detection statistics
    motion_stats = []

    # Read the first frame
    ret, frame1 = cap.read()
    if not ret:
        print("Error: Could not read the first frame.")
        return

    # Convert the first frame to grayscale
    gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    gray1 = cv2.GaussianBlur(gray1, (21, 21), 0)

    while cap.isOpened():
        ret, frame2 = cap.read()
        if not ret:
            break

        # Convert the current frame to grayscale
        gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
        gray2 = cv2.GaussianBlur(gray2, (21, 21), 0)

        # Compute the absolute difference between the first frame and current frame
        diff = cv2.absdiff(gray1, gray2)

        # Apply a threshold to get the binary image
        _, thresh = cv2.threshold(diff, 25, 255, cv2.THRESH_BINARY)

        # Dilate the threshold image to fill in holes, then find contours
        thresh = cv2.dilate(thresh, None, iterations=2)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Draw the contours on the frame and collect statistics
        for contour in contours:
            if cv2.contourArea(contour) < 500:
                continue  # Ignore small contours
            (x, y, w, h) = cv2.boundingRect(contour)
            cv2.rectangle(frame2, (x, y), (x + w, y + h), (0, 255, 0), 2)

            # Calculate the timestamp of the current frame
            frame_number = cap.get(cv2.CAP_PROP_POS_FRAMES)
            timestamp = frame_number / fps

            # Collect the motion detection event statistics
            motion_stats.append({
                "timestamp": timestamp,
                "rectangle_size": w * h
            })

        # Write the frame into the output video
        out.write(frame2)

        # Update the first frame
        gray1 = gray2

    # Release the video capture and writer objects
    cap.release()
    out.release()

    # Save the motion statistics to a CSV file
    motion_stats_df = pd.DataFrame(motion_stats)
    motion_stats_df.to_csv(stats_output_path, index=False)

In [8]:
video_path = "videos/raw_2024-07-15_21-04-35.mp4"  # Replace with your video file path
output_path = video_path.replace("raw", "motion_raw")
stats_output_path = video_path.replace(
    "videos/", "motion_stats/motion_stats_").replace(".mp4", ".csv")

In [9]:
main(video_path, output_path, stats_output_path)